# Pacotes

In [1]:
from deltalake import DeltaTable, write_deltalake
import mlflow
from mlflow.models import infer_signature
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import shap
import Funcoes
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score, average_precision_score, brier_score_loss, confusion_matrix, classification_report, ConfusionMatrixDisplay, precision_recall_curve
import xgboost as xgb

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Objetivo

Primeiro, algumas variáveis serão descartadas pelo acúmulo de importância (~90%) e pelo valor de IV.

Segundo, com o uso da validação cruzada: será avaliado o comportamento médio de algumas métricas calculadas nos modelos baselines como a curva sobre a curva precision e recall (AUCPR) e Brier Score (BS); foi visto que a identificação de um melhor ponto de corte pode melhorar a métrica F1-score. Dessa forma, será calculado qual o melhor ponto de corte médio.

Terceiro, será escolhido qual o melhor conjunto de hiperparâmetros para os modelos Random Forest e XGBoost.

# Leitura da base v1 e filtro de variáveis

Variáveis com valor acumulado de importância em 90% e pelo valor de IV.

In [6]:
# A chance de ser ou não churn não depende do sexo ser M ou F. Mas, ela pode ser uma proxy para comportamentos socioeconômicos como diferença de renda, ocupação em 
# atividades não remuneradas (múltiplas jornadas de trabalho) etc

# Estado civil e número de dependentes podem ser interpretados de forma semelhante. Ambas podem direcionar para um maior compromisso com a estabilidade financeira,
# pessoas com mais idade e, possivelmente, com principalidade de uso do cartão já definida.

dados = DeltaTable("../1.Variaveis/tmp/dados_pp_v1").to_pandas()
dados.drop(['__index_level_0__', 'Card_Category_Gold', 'Card_Category_Platinum', 'Card_Category_Silver', 
            'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Unknown', 'Gender_M',
            'Dependent_count'], axis=1, inplace=True)
dados.head()

,Customer_Age,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Ct_Chng_Q4_Q1,vfm,pmcc,Income_Category_1.< 40k,Income_Category_2. >= 40k & < 60k,Income_Category_3. >= 60k & < 80k,Income_Category_4. >= 80k & < 120k,Income_Category_5. >= 120k,Education_Level_v2_1.Uneducated,Education_Level_v2_2.High School,Education_Level_v2_3.Graduate,Education_Level_v2_4.Post-Graduate,Attrition_Flag,type
0,-0.784196,1.403132,-1.337898,0.498943,0.963894,0.282975,-0.328225,-0.175537,-0.421450,1,0,0,0,0,0,0,1,0,0,Treino
1,0.720070,-0.525933,0.641818,1.408428,-0.165769,-1.527806,-0.194304,-0.208685,-1.054789,0,0,0,1,0,0,0,1,0,0,Treino
2,1.346848,-0.525933,-0.348040,0.498943,0.864865,0.894171,0.056797,-0.571459,-0.686436,0,0,0,1,0,0,0,0,1,0,Treino
3,0.218648,-0.525933,0.641818,-1.320028,-0.412731,0.369637,0.851953,0.252749,2.406712,1,0,0,0,0,0,0,1,0,0,Treino
4,-2.539173,0.117089,1.631675,1.408428,-0.858972,0.346832,-1.144306,-0.064053,-0.071911,1,0,0,0,0,1,0,0,0,0,Treino


In [7]:
dados.duplicated().sum()

np.int64(0)

In [8]:
dados.shape

(10127, 20)

# Separação das bases

In [ ]:
dados_treino = dados[dados.type == 'Treino'].drop(['type'], axis=1)
dados_val = dados[dados.type == 'Validacao'].drop(['type'], axis=1)
dados_teste = dados[dados.type == 'Teste'].drop(['type'], axis=1)

In [ ]:
X_treino = dados_treino.drop(['Attrition_Flag'], axis=1)
y_treino = dados_treino['Attrition_Flag']

X_val = dados_val.drop(['Attrition_Flag'], axis=1)
y_val = dados_val['Attrition_Flag']

https://scikit-learn.org/1.5/modules/cross_validation.html